# Red Neuronal Profunda (DNN) para clasificación MNIST

Aplicaremos todos nuestros conocimientos para crear una DNN, frecuentemente llamada también una Artificial Neural Network (ANN).  El problema que vamos a trabajar se conoce como el "Hola Mundo" del aprendizaje profundo porque para la mayoría de estudiantes este es el primer algoritmo de aprendizaje profundo que ven.

El conjunto de datos se llama MNIST y se refiere al reconocimiento de dígitos escritos a mano.  Pueden encontrar más información en el sitio web de Yann LeCun (Director of AI Research, Facebook).  El es uno de los pioneros de todo este tema, así como de otras metodologías más complejas como las Redes Neurales Convolucionales (CNN) que se utilizan hoy día.

El conjunto de datos tiene 70,000 imágenes (28x28 pixels) de dígitos escritos a mano (1 dígito por imagen).

La meta es escribir un algoritmo que detecta qué dígito ha sido escrito.  Como solo hay 10 dígitos (0 al 9), este es un problema de clasificación con 10 clases.

Nuestra meta será construir una RN con 2 capas escondidas.

## Plan de Acción para preparar el modelo

1.  Preparar los datos y preprocesarlos.  Crear los conjuntos de datos para entrenar, validar y probar
2.  Crear un esboso del modelo y seleccionar las funciones de activación
3.  Fijar los optimizadores avanzados y la función de pérdida
4.  Hacer que el modelo aprenda
5.  Probar la exactitud ("accuracy") del modelo

## Importar los paquetes relevantes

TensorFlow incluye un proveedor de los datos de MNIST que utilizaremos acá.  Viene con el módulo **"tensorflow.keras.datasets"**.

In [76]:
import numpy as np
import tensorflow as tf

La siguiente instrucción, cuando se corre por primera vez, descarga el conjunto de datos en lo indicado por el parámetro path, relativo a  ~/.keras/datasets).  Como si se hubiera ejecutado Lo siguiente:

tf.keras.datasets.mnist.load_data(
    path = 'mnist.npz'
)

luego separa los datos en un conjunto para entrenamiento y otro para pruebas.

Si se ejecuta más de una vez, ya no descarga el archivo.

In [77]:
(X_entreno, y_entreno), (X_prueba, y_prueba) = tf.keras.datasets.mnist.load_data()


In [78]:
X_entreno

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

Como no podemos ver la forma de los conjuntos...les queda de tarea averiguar por qué no...podemos utilizar la instrucción **assert**

In [79]:
assert X_entreno.shape == (60000, 28, 28)
assert X_prueba.shape == (10000, 28, 28)
assert y_entreno.shape == (60000,)
assert y_prueba.shape == (10000,)

## Datos

Esta sección es donde pre-procesaremos nuestros datos.

Por default, TF2 tiene conjuntos de datos de entrenamiento y de prueba, pero no tiene un conjunto de validación, por lo que debemos dividirlo por nuestra cuenta

Lo haremos del mismo tamaño que el conjunto de prueba

In [80]:
num_obs_validacion = y_prueba.shape[0]

Usaremos una variable dedicada para el número de observaciones de prueba

In [81]:
num_obs_prueba = y_prueba.shape[0]

Generalmente preferimos "normalizar" nuestros datos en alguna forma para que el resultado sea numéricamente más estable.  En este caso simplemente preferimos tener entradas entre 0 y 1, por lo que definimos una función, que reciba la imagen MNIST.

Como los posibles valores de las entradas son entre 0 y 255 (256 posibles tonos de gris), al dividirlos por 255 obtenemos el resultado deseado.

In [82]:
X_entreno_normalizado = X_entreno / 255

Finalmente, normalizaremos y convertiremos los datos de pruebas en tandas.  Los normalizamos para que tengan la misma magnitud que los datos de entrenamiento y validación.

No hay necesidad de "barajearlo" ya que no estaremos entrenando con los datos de prueba.  Habra una sola tanda, igual al tamaño de los datos de prueba.

In [83]:
X_prueba_normalizado = X_prueba / 255

Una vez se han "normalizado" los datos, podemos proceder a extraer los datos de entrenamiento y de validación.

Nuestros datos de validación serán 10000 para ser igual al conjunto de prueba.

Finalmente, creamos una tanda con un tamaño de tanda igual al total de muestras de validación.

In [84]:
X_validacion = X_entreno_normalizado[-num_obs_validacion: , : , : ]
y_validacion = y_entreno[-num_obs_validacion:]

Similarmente, los datos de entrenamiento son todos los demás por lo que nos salteamos tantas observaciones como las hay en el conjunto de validación.

In [85]:
X_entreno = X_entreno_normalizado[ : X_entreno_normalizado.shape[0] - num_obs_validacion, : , : ]
y_entreno = y_entreno[ : y_entreno.shape[0] - num_obs_validacion]
num_obs_entreno = y_entreno.shape[0]

Convertir de Arreglos Numpy a Tensores

In [86]:
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

Barajear y hacer tandas con el conjunto de datos de entrenamiento

In [87]:
TAMANIO_TANDA = 100
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)

Hacer tandas con los conjuntos de validación y prueba, no se necesita barajearlos

In [88]:
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

## Modelo

### Delineamos el modelo

Cuando pensamos sobre un algoritmo de aprendizaje profundo, casi siempre imaginamos la realización del mismo.  Asi que esta vez, hagámoslo.  :)

In [89]:
tamanio_entrada = 784
tamanio_salida = 10

Usaremos el mismo ancho para ambas capas escondidas.  (No es una necesidad!)

In [90]:
tamanio_capa_escondida = 50

# Definimos cómo se verá el modelo

La primera capa (la de entrada):  cada observación es de 28x28 píxeles, por lo tanto es un tensor de rango 2.

Como aún no hemos aprendido sobre CNNs, no sabemos como alimentar este tipo de entrada a nuestra red, por lo tanto hay que "aplanar" las imágenes.  Hay un método conveniente **Flatten** que toma nuestro tensor de 28x28 y lo convierte en  un vector (None,) o (784,)...porque 28x28 = 784.  Esto nos permite crear una red de alimentación hacia adelante.

    
**tf.keras.layers.Dense** básicamente implementa:  *salida = activation(dot(entrada, peso) + sesgo)*.  Requiere varios argumentos, pero los más importantes para nosotros son el ancho de la capa escondida y la función de activación.

La capa final no es diferente, solo nos aseguramos de activarla con **softmax**


In [91]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

### Seleccionar el optimizador y la función de pérdida

Definimos el optimizador que nos gustaría utilizar, la función de pérdida, y las métricas que nos interesa obtener en cada interacción

In [92]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Entrenamiento

Acá es donde entrenamos el modelo que hemos construído

Determinamos el número máximo de épocas.

Ajustamos el modelo , especificando:

* los datos de entrenamiento
* el número total de épocas
* y los datos de validación que creamos en el formato (entradas, metas)

In [93]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

Epoch 1/5
500/500 - 3s - 7ms/step - accuracy: 0.8774 - loss: 0.4387 - val_accuracy: 0.9475 - val_loss: 0.1879
Epoch 2/5
500/500 - 1s - 3ms/step - accuracy: 0.9473 - loss: 0.1801 - val_accuracy: 0.9576 - val_loss: 0.1510
Epoch 3/5
500/500 - 1s - 3ms/step - accuracy: 0.9602 - loss: 0.1358 - val_accuracy: 0.9600 - val_loss: 0.1339
Epoch 4/5
500/500 - 2s - 4ms/step - accuracy: 0.9665 - loss: 0.1133 - val_accuracy: 0.9651 - val_loss: 0.1219
Epoch 5/5
500/500 - 1s - 3ms/step - accuracy: 0.9702 - loss: 0.0956 - val_accuracy: 0.9671 - val_loss: 0.1171


## Probar el modelo

Como se discutió en clase, luego del entrenamiento (con los datos de entrenamiento), y la validación (con los datos de validación), probamos el potencial de predicción final de nuestro modelo con el conjunto de datos de prueba que el algoritmo NUNCA ha visto antes.

Es muy importante reconocer que estar "jugando" con los hiperparámetros sobre-ajusta el conjunto de datos de validación.

La prueba es la instancia absolutamente final. **NUNCA** debe probarse el modelo antes de haber completamente ajustado el mismo.

Si se ajusta el modelo después de hacer la prueba, se empezará a sobre-ajustar el conjunto de datos de prueba, que echaría "por los suelos" el propósito original del mismo.

In [94]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9614 - loss: 20.6999


In [95]:
# Si se desea, se puede aplicar un formateo "bonito"
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Pérdida de prueba: 18.33. Precisión de prueba: 96.60%


Utilizando el modelo inicial y los hiperparámetros dados en este notebook, la precisión de prueba final debe ser aproximadamente 97%.

Cada vez que se ejecuta el código, se obtiene una precisión diferente debido a la "barajeada" de las tandas, los pesos se inicializan en forma diferente, etc.

Finalmente, intencionalmente se ha llegado a una solución subóptima, para que puedan tener la oportunidad de mejorarla como ejercicio de laboratorio.

# 1. Modificación Ancho de la Red
Para este inciso estaremos trabajando sobre el modelo Baseline proporcionado, teniendo dos capas "baseline" utilizando diferentes tamaños de capas escondidas.

## 200 Neuronas

In [96]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(200, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(200, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 3s - 6ms/step - accuracy: 0.9179 - loss: 0.2862 - val_accuracy: 0.9625 - val_loss: 0.1349
Epoch 2/5
500/500 - 1s - 3ms/step - accuracy: 0.9672 - loss: 0.1100 - val_accuracy: 0.9712 - val_loss: 0.1001
Epoch 3/5
500/500 - 1s - 3ms/step - accuracy: 0.9784 - loss: 0.0717 - val_accuracy: 0.9688 - val_loss: 0.1047
Epoch 4/5
500/500 - 1s - 3ms/step - accuracy: 0.9836 - loss: 0.0537 - val_accuracy: 0.9761 - val_loss: 0.0810
Epoch 5/5
500/500 - 2s - 4ms/step - accuracy: 0.9867 - loss: 0.0403 - val_accuracy: 0.9761 - val_loss: 0.0808
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9726 - loss: 15.2571
Pérdida de prueba: 12.85. Precisión de prueba: 97.77%


## 50 Neuronas

In [97]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(50, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(50, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 3s - 6ms/step - accuracy: 0.8801 - loss: 0.4233 - val_accuracy: 0.9356 - val_loss: 0.2208
Epoch 2/5
500/500 - 1s - 3ms/step - accuracy: 0.9438 - loss: 0.1963 - val_accuracy: 0.9525 - val_loss: 0.1695
Epoch 3/5
500/500 - 3s - 5ms/step - accuracy: 0.9554 - loss: 0.1484 - val_accuracy: 0.9565 - val_loss: 0.1455
Epoch 4/5
500/500 - 1s - 3ms/step - accuracy: 0.9645 - loss: 0.1200 - val_accuracy: 0.9625 - val_loss: 0.1272
Epoch 5/5
500/500 - 1s - 3ms/step - accuracy: 0.9691 - loss: 0.1004 - val_accuracy: 0.9655 - val_loss: 0.1199
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9569 - loss: 21.2703
Pérdida de prueba: 19.84. Precisión de prueba: 96.10%


## 100 Neuronas

In [98]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(100, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(100, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 3s - 6ms/step - accuracy: 0.9002 - loss: 0.3501 - val_accuracy: 0.9544 - val_loss: 0.1582
Epoch 2/5
500/500 - 1s - 3ms/step - accuracy: 0.9571 - loss: 0.1438 - val_accuracy: 0.9668 - val_loss: 0.1171
Epoch 3/5
500/500 - 1s - 3ms/step - accuracy: 0.9700 - loss: 0.0992 - val_accuracy: 0.9680 - val_loss: 0.1122
Epoch 4/5
500/500 - 1s - 3ms/step - accuracy: 0.9770 - loss: 0.0753 - val_accuracy: 0.9734 - val_loss: 0.0885
Epoch 5/5
500/500 - 1s - 3ms/step - accuracy: 0.9816 - loss: 0.0600 - val_accuracy: 0.9732 - val_loss: 0.0910
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9696 - loss: 18.2785
Pérdida de prueba: 16.22. Precisión de prueba: 97.24%


## 300 Neuronas

In [99]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(300, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(300, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 3s - 7ms/step - accuracy: 0.9235 - loss: 0.2564 - val_accuracy: 0.9643 - val_loss: 0.1173
Epoch 2/5
500/500 - 2s - 3ms/step - accuracy: 0.9707 - loss: 0.0963 - val_accuracy: 0.9721 - val_loss: 0.0863
Epoch 3/5
500/500 - 2s - 5ms/step - accuracy: 0.9812 - loss: 0.0616 - val_accuracy: 0.9763 - val_loss: 0.0790
Epoch 4/5
500/500 - 1s - 3ms/step - accuracy: 0.9862 - loss: 0.0437 - val_accuracy: 0.9767 - val_loss: 0.0797
Epoch 5/5
500/500 - 1s - 3ms/step - accuracy: 0.9905 - loss: 0.0307 - val_accuracy: 0.9775 - val_loss: 0.0813
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9729 - loss: 18.3866
Pérdida de prueba: 14.86. Precisión de prueba: 97.77%


## 500 Neuronas

In [100]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(500, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(500, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 3s - 7ms/step - accuracy: 0.9315 - loss: 0.2292 - val_accuracy: 0.9684 - val_loss: 0.1054
Epoch 2/5
500/500 - 2s - 3ms/step - accuracy: 0.9731 - loss: 0.0868 - val_accuracy: 0.9721 - val_loss: 0.0950
Epoch 3/5
500/500 - 1s - 3ms/step - accuracy: 0.9827 - loss: 0.0529 - val_accuracy: 0.9732 - val_loss: 0.0912
Epoch 4/5
500/500 - 1s - 3ms/step - accuracy: 0.9889 - loss: 0.0362 - val_accuracy: 0.9751 - val_loss: 0.0920
Epoch 5/5
500/500 - 2s - 3ms/step - accuracy: 0.9900 - loss: 0.0301 - val_accuracy: 0.9790 - val_loss: 0.0758
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9738 - loss: 16.2990
Pérdida de prueba: 13.89. Precisión de prueba: 97.69%


## 2. Modificación de la Profundidad de Red

In [101]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(500, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(500, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(500, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 4s - 7ms/step - accuracy: 0.9312 - loss: 0.2283 - val_accuracy: 0.9609 - val_loss: 0.1337
Epoch 2/5
500/500 - 4s - 8ms/step - accuracy: 0.9722 - loss: 0.0889 - val_accuracy: 0.9739 - val_loss: 0.0869
Epoch 3/5
500/500 - 3s - 5ms/step - accuracy: 0.9816 - loss: 0.0599 - val_accuracy: 0.9737 - val_loss: 0.0881
Epoch 4/5
500/500 - 2s - 3ms/step - accuracy: 0.9861 - loss: 0.0446 - val_accuracy: 0.9684 - val_loss: 0.1133
Epoch 5/5
500/500 - 2s - 3ms/step - accuracy: 0.9886 - loss: 0.0360 - val_accuracy: 0.9802 - val_loss: 0.0706
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9776 - loss: 16.4733
Pérdida de prueba: 13.43. Precisión de prueba: 98.08%


## 3. Redes Profundas

In [102]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(500, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(500, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(500, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(500, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 5s - 9ms/step - accuracy: 0.9291 - loss: 0.2295 - val_accuracy: 0.9687 - val_loss: 0.1062
Epoch 2/5
500/500 - 2s - 4ms/step - accuracy: 0.9699 - loss: 0.0961 - val_accuracy: 0.9717 - val_loss: 0.0965
Epoch 3/5
500/500 - 2s - 3ms/step - accuracy: 0.9802 - loss: 0.0645 - val_accuracy: 0.9756 - val_loss: 0.0837
Epoch 4/5
500/500 - 2s - 3ms/step - accuracy: 0.9839 - loss: 0.0515 - val_accuracy: 0.9764 - val_loss: 0.0889
Epoch 5/5
500/500 - 2s - 3ms/step - accuracy: 0.9872 - loss: 0.0419 - val_accuracy: 0.9727 - val_loss: 0.0970
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9680 - loss: 21.4640
Pérdida de prueba: 17.74. Precisión de prueba: 97.19%


In [103]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(500, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(500, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(500, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(500, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(500, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 5s - 11ms/step - accuracy: 0.9252 - loss: 0.2442 - val_accuracy: 0.9672 - val_loss: 0.1138
Epoch 2/5
500/500 - 2s - 4ms/step - accuracy: 0.9682 - loss: 0.1058 - val_accuracy: 0.9688 - val_loss: 0.1007
Epoch 3/5
500/500 - 2s - 4ms/step - accuracy: 0.9781 - loss: 0.0745 - val_accuracy: 0.9760 - val_loss: 0.0863
Epoch 4/5
500/500 - 2s - 4ms/step - accuracy: 0.9832 - loss: 0.0567 - val_accuracy: 0.9755 - val_loss: 0.1004
Epoch 5/5
500/500 - 2s - 4ms/step - accuracy: 0.9862 - loss: 0.0486 - val_accuracy: 0.9734 - val_loss: 0.1007
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9668 - loss: 19.3245
Pérdida de prueba: 16.50. Precisión de prueba: 97.36%


## 4. Funciones de Activación I

In [104]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 4s - 7ms/step - accuracy: 0.7790 - loss: 1.0551 - val_accuracy: 0.9036 - val_loss: 0.4186
Epoch 2/5
500/500 - 2s - 4ms/step - accuracy: 0.9110 - loss: 0.3453 - val_accuracy: 0.9302 - val_loss: 0.2604
Epoch 3/5
500/500 - 1s - 3ms/step - accuracy: 0.9295 - loss: 0.2509 - val_accuracy: 0.9416 - val_loss: 0.2112
Epoch 4/5
500/500 - 1s - 3ms/step - accuracy: 0.9404 - loss: 0.2075 - val_accuracy: 0.9472 - val_loss: 0.1860
Epoch 5/5
500/500 - 1s - 3ms/step - accuracy: 0.9474 - loss: 0.1793 - val_accuracy: 0.9510 - val_loss: 0.1693
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9348 - loss: 0.2195
Pérdida de prueba: 0.20. Precisión de prueba: 94.12%


## 5. Funciones de Activación II

In [105]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='tanh'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 4s - 8ms/step - accuracy: 0.8832 - loss: 0.4246 - val_accuracy: 0.9424 - val_loss: 0.2057
Epoch 2/5
500/500 - 1s - 3ms/step - accuracy: 0.9450 - loss: 0.1881 - val_accuracy: 0.9566 - val_loss: 0.1574
Epoch 3/5
500/500 - 3s - 5ms/step - accuracy: 0.9583 - loss: 0.1408 - val_accuracy: 0.9636 - val_loss: 0.1291
Epoch 4/5
500/500 - 1s - 3ms/step - accuracy: 0.9660 - loss: 0.1113 - val_accuracy: 0.9636 - val_loss: 0.1196
Epoch 5/5
500/500 - 2s - 5ms/step - accuracy: 0.9721 - loss: 0.0921 - val_accuracy: 0.9680 - val_loss: 0.1058
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9556 - loss: 0.1601
Pérdida de prueba: 0.14. Precisión de prueba: 96.01%


## 6. Tamaño de Batch Grande

In [106]:
TAMANIO_TANDA = 10000

datos_entreno = datos_entreno.unbatch().shuffle(buffer_size=num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.unbatch().batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.unbatch().batch(TAMANIO_TANDA)


In [107]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
5/5 - 3s - 636ms/step - accuracy: 0.2621 - loss: 2.1966 - val_accuracy: 0.4190 - val_loss: 2.0405
Epoch 2/5
5/5 - 2s - 306ms/step - accuracy: 0.4726 - loss: 1.9281 - val_accuracy: 0.5234 - val_loss: 1.7512
Epoch 3/5
5/5 - 1s - 251ms/step - accuracy: 0.5565 - loss: 1.6364 - val_accuracy: 0.6092 - val_loss: 1.4545
Epoch 4/5
5/5 - 1s - 163ms/step - accuracy: 0.6406 - loss: 1.3539 - val_accuracy: 0.7042 - val_loss: 1.1757
Epoch 5/5
5/5 - 1s - 251ms/step - accuracy: 0.7170 - loss: 1.0987 - val_accuracy: 0.7788 - val_loss: 0.9310
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step - accuracy: 0.7648 - loss: 40.3152
Pérdida de prueba: 40.32. Precisión de prueba: 76.48%


## 7. Descenso de Gradiente Estocástico

In [108]:
TAMANIO_TANDA = 1

datos_entreno = datos_entreno.unbatch().shuffle(buffer_size=num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.unbatch().batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.unbatch().batch(TAMANIO_TANDA)

In [109]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
50000/50000 - 118s - 2ms/step - accuracy: 0.9230 - loss: 0.2595 - val_accuracy: 0.9529 - val_loss: 0.1698
Epoch 2/5
50000/50000 - 141s - 3ms/step - accuracy: 0.9558 - loss: 0.1582 - val_accuracy: 0.9623 - val_loss: 0.1336
Epoch 3/5
50000/50000 - 142s - 3ms/step - accuracy: 0.9623 - loss: 0.1389 - val_accuracy: 0.9418 - val_loss: 0.2180
Epoch 4/5
50000/50000 - 115s - 2ms/step - accuracy: 0.9657 - loss: 0.1306 - val_accuracy: 0.9656 - val_loss: 0.1424
Epoch 5/5
50000/50000 - 144s - 3ms/step - accuracy: 0.9685 - loss: 0.1294 - val_accuracy: 0.9590 - val_loss: 0.1660
Pérdida de prueba: 40.32. Precisión de prueba: 76.48%


## 8. Tasa de Aprendizaje Baja

In [110]:
TAMANIO_TANDA = 100

datos_entreno = datos_entreno.unbatch().shuffle(buffer_size=num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.unbatch().batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.unbatch().batch(TAMANIO_TANDA)

modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

optimizador = tf.keras.optimizers.Adam(learning_rate=0.0001)

modelo.compile(optimizer=optimizador, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 4s - 9ms/step - accuracy: 0.6607 - loss: 1.2952 - val_accuracy: 0.8722 - val_loss: 0.5756
Epoch 2/5
500/500 - 4s - 9ms/step - accuracy: 0.8782 - loss: 0.4806 - val_accuracy: 0.9066 - val_loss: 0.3636
Epoch 3/5
500/500 - 10s - 20ms/step - accuracy: 0.9015 - loss: 0.3615 - val_accuracy: 0.9174 - val_loss: 0.3044
Epoch 4/5
500/500 - 10s - 20ms/step - accuracy: 0.9128 - loss: 0.3146 - val_accuracy: 0.9226 - val_loss: 0.2758
Epoch 5/5
500/500 - 3s - 5ms/step - accuracy: 0.9196 - loss: 0.2863 - val_accuracy: 0.9289 - val_loss: 0.2555
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9118 - loss: 37.6938
Pérdida de prueba: 33.43. Precisión de prueba: 92.57%


## 9. Tasa de Aprendizaje Alta

In [111]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

optimizador = tf.keras.optimizers.Adam(learning_rate=0.02)

modelo.compile(optimizer=optimizador, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 5s - 10ms/step - accuracy: 0.9044 - loss: 0.3141 - val_accuracy: 0.9514 - val_loss: 0.1783
Epoch 2/5
500/500 - 3s - 5ms/step - accuracy: 0.9407 - loss: 0.2071 - val_accuracy: 0.9402 - val_loss: 0.2151
Epoch 3/5
500/500 - 5s - 10ms/step - accuracy: 0.9487 - loss: 0.1849 - val_accuracy: 0.9490 - val_loss: 0.1800
Epoch 4/5
500/500 - 3s - 6ms/step - accuracy: 0.9543 - loss: 0.1655 - val_accuracy: 0.9575 - val_loss: 0.1707
Epoch 5/5
500/500 - 5s - 9ms/step - accuracy: 0.9568 - loss: 0.1612 - val_accuracy: 0.9453 - val_loss: 0.2446
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9367 - loss: 51.0832
Pérdida de prueba: 47.56. Precisión de prueba: 94.24%


## 10. Optimización Avanzada

## 11. Visualización

## 12. Modelo Óptimo